In [ ]:
import sys
import os
sys.path = [p for p in sys.path if "ParaView" not in p]
sys.path.append("/home/haseeb/Notebooks/Parameteric_DMD_cylinder_2D_modules")

import math
import torch as pt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as mtri
from matplotlib.patches import Circle 
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from flowtorch import DATASETS
from flowtorch.data import FOAMDataloader, mask_box
from flowtorch.analysis import DMD
from ezyrb import POD, Database, RBF, GPR, Linear
import seaborn as sns
from sklearn.utils.extmath import randomized_svd
from scipy.interpolate import griddata

from Flowtorch_DMD_cylinder_2D_modules import load_all_snapshots, preprocess_snapshots



from pathlib import Path
import re
# For High Quality Visuals
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 160  # crisp visuals

## Step 1. Defining training and test parameters

We define a training set *P*, for the partitioned Dynamic Mode Decomposition as:
$$
P = \{\mu_1, \mu_2, \dots, \mu_P\}, \quad 
\mu_j \equiv Re_j $$

where
- $P$ : set of training parameters which are Reynolds numbers $Re$ in this case. 
- $\mu_j$ : $j$‑th training parameter in $P$ 

And, the unseen parameter is given as $\mu^\star$, for which the spectral information ($\lambda$, $\phi$, $b$) of the forecasted DMD from the training set $P$ is interpolated.

$$
\mu^\star \equiv Re^\star
$$

where 
- $\lambda_j^{(\mu_j)}$ : DMD eigenvalue associated with mode $j$ at parameter $\mu_j$
- $\phi_j^{(\mu_j)}$: $j$‑th DMD mode vector  
- $Re^\star$: test Reynolds number
- $b_{jk}^{(\mu_j)}$: amplitude of mode $j$ at time $k$


In [ ]:
# Training and test parameters (parameter set P and test parameter μ*)

# Training parameter set P = {μ1, ..., μP}, i.e., Reynolds numbers
Re_list = np.array([100, 110, 120, 130, 140, 150, 160, 170, 180, 200])  # μ: training parameters

# Test parameter μ* (unseen Reynolds number)
Re_test = 190  # μ*: test parameter
# Visualization of parameter set P and test parameter μ*
fig, ax = plt.subplots(figsize=(10, 3))
plt.title('Parameter set $P$', fontsize=14)

# Plot all training parameters (μ)
ax.scatter(Re_list, np.zeros_like(Re_list), 
           marker='o', s=80, color='royalblue', edgecolor='white', 
           label='Train ($μ$)', zorder=3)

# Plot test parameter (μ*)
ax.scatter(Re_test, 0, 
           marker='D', s=120, color='darkorange', edgecolor='black', 
           label='Test ($μ*$)', zorder=4)

# Formatting
plt.xticks(np.append(Re_list, Re_test), rotation=45, fontsize=10)
plt.yticks([])
ax.axhline(0, color='lightgray', linewidth=1.5, zorder=1)   # baseline line
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.legend(ncols=2, frameon=True, fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
# Load and prepare data (velocity components Ux, Uy)
import os
from Flowtorch_DMD_cylinder_2D_modules import load_snapshots

# Base folder containing simulation data
base_path = os.path.expanduser("~/OpenFOAM/test_1")

# Time windows
training_window = (10.0, 15.0)  # training window T: snapshots used to build X^(μ)
future_window   = (15.0, 20.0)  # forecast window T_future: snapshots used for validation

# Sampling step for downsampling time steps
sampling_step = 1


# Load training and future snapshots for all training parameters μ ∈ P
# (velocity only: [Ux; Uy])

data = load_snapshots.load_all_snapshots(
    Re_list=Re_list,                 # P: training parameter set
    base_path=base_path,
    mask_box=mask_box,
    FOAMDataloader=FOAMDataloader,
    training_window=training_window, # T: training time instants
    future_window=future_window,     # T_future: forecast time instants
    sampling_step=sampling_step
)

# Access training data
snapshot_dict             = data["snapshot_dict"]              # X^(μ): stacked [Ux; Uy] snapshots
sampled_times_dict        = data["sampled_times_dict"]         # T: training time instants
snapshot_future_dict      = data["snapshot_future_dict"]       # X_future^(μ): stacked future snapshots
sampled_times_future_dict = data["sampled_times_future_dict"]  # T_future: forecast time instants
masked_coords_dict        = data["masked_coords_dict"]         # spatial coordinates after masking
num_points_dict           = data["num_points_dict"]            # number of spatial points after masking


# Load test snapshots for unseen parameter p* (forecast-aligned only)
# (velocity only: [Ux; Uy])
path_test = f"{base_path}/cylinder_2D_Re{Re_test}"
test_data = load_snapshots.load_test_parameter(
    Re=Re_test,                       # p*: test parameter
    path=path_test,
    mask_box=mask_box,
    FOAMDataloader=FOAMDataloader,
    forecast_window=future_window,    # T_future: forecast time instants
    sampling_step=sampling_step
)

# Access test forecast data
snapshot_test_future      = test_data["snapshot_forecast"]       # X_future^(p*): stacked [Ux; Uy] forecast snapshots
sampled_times_test_future = test_data["sampled_times_forecast"]  # T_future: forecast time instants for test p*
mask_test                 = test_data["mask"]
coords_test               = test_data["masked_coords"]
num_points_test           = test_data["num_points"]
loader_test               = test_data["loader"]




In [ ]:
# Snapshot magnitude over time for each Parameter i.e., Reynolds number

from Flowtorch_DMD_cylinder_2D_modules.vis_util import plot_snapshot_magnitudes

loader_dict = data["loader_dict"]  

plot_snapshot_magnitudes(snapshot_dict, sampled_times_dict, Re_list)


## Step 2. Train partitioned DMD

For each training parameter $\mu_j \in P = \{\mu_1, \mu_2, \dots, \mu_p\}$ i.e., Reynolds numbers $Re$, the fitted DMD operator is obtained via

$$
A^{(\mu_j)} \;\; \approx \;\; Y^{(\mu_j)} \, \big(X^{(\mu_j)})^{\dagger}
$$


where  
- $\mu_j$ : $j$‑th training parameter in the set $S$  
- $Re$ : Reynolds number corresponding to $\mu_j$  
- $X^{(\mu_j)} \in \mathbb{R}^{n_{\text{space}} \times (n_{\text{time}}-1)}$ : snapshot matrix at times $t_1, \dots, t_{n_{\text{time}}-1}$  
- $Y^{(\mu_j)} \in \mathbb{R}^{n_{\text{space}} \times (n_{\text{time}}-1)}$ : shifted snapshot matrix at times $t_2, \dots, t_{n_{\text{time}}}$  
- $A^{(\mu_j)}$ : DMD regression operator advancing the reduced state forward in time  



In [ ]:
dmd_dict = {}

for Re in Re_list:
    # Snapshot matrix X^(p) ∈ ℝ^(space_dim × n_time)
    snapshots_np = snapshot_dict[Re]
    X_p = pt.tensor(snapshots_np, dtype=pt.float64)

    # Fit DMD models for each training parameter i.e., Reynolds number

    # - tlsq=True → Total Least Squares DMD (TDMD, Hemati et al.)
    # - unitary=True → piDMD (unitary operator constraint, Baddoo)
    # - optimal=True → spDMD (optimal amplitudes via LSQ, Janovic et al.)
    # - forward_backward=True → CDMD (forward/backward consistency, Azencot et al.)
    

    # Operator A ≈ Y X^+ where:
    dmd = DMD(X_p, dt=0.01, rank=30, tlsq=False, unitary=False, optimal=False, forward_backward=False)

    # Store the DMD object for this Reynolds number
    dmd_dict[int(Re)] = dmd

    print(f"Re = {Re}", dmd)


In [ ]:
# Plot DMD spatial modes for training parameter i.e., Reynolds number

Re_target = 130

# Retrieve DMD object directly
dmd_model = dmd_dict[Re_target]

# DMD spatial modes Φ ∈ ℂ^(2*num_points × r)
modes = dmd_model.modes.detach().cpu().numpy()

# Masked coordinates (n_points × 2) from loader
coords = data["masked_coords_dict"][Re_target]
num_points = data["num_points_dict"][Re_target]

# Select top-k modes by amplitude
topk = 6
modes_to_plot = dmd_model.top_modes(n=topk, integral=False).cpu().numpy()

# Compute global min/max across all selected modes
all_mags = []
for m in modes_to_plot:
    mode_u = modes[:num_points, m].real
    mode_v = modes[num_points:, m].real
    mode_mag = np.sqrt(mode_u**2 + mode_v**2)
    all_mags.append(mode_mag)

global_min = min(np.min(m) for m in all_mags)
global_max = max(np.max(m) for m in all_mags)

# Arrange plots in 2 columns
ncols = 2
nrows = int(np.ceil(topk / ncols))

fig, axarr = plt.subplots(nrows, ncols, figsize=(10, 3*nrows), sharex=True, sharey=True)
axarr = axarr.flatten()

for i, (m, mode_mag) in enumerate(zip(modes_to_plot, all_mags)):
    sc = axarr[i].tricontourf(
        coords[:, 0], coords[:, 1], mode_mag,
        levels=500, cmap="icefire",
        vmin=global_min, vmax=global_max
    )

    axarr[i].add_patch(Circle((0.2, 0.2), 0.05, color='black', zorder=10))
    axarr[i].set_title(f"Mode $\Phi_{{{m}}}$", fontsize=14)
    axarr[i].set_aspect("equal")
    axarr[i].axis("off")

# Hide unused subplots if topk is not multiple of ncols
for j in range(len(modes_to_plot), len(axarr)):
    axarr[j].axis("off")

# Single shared colorbar at bottom 
norm = Normalize(vmin=global_min, vmax=global_max)
sm = ScalarMappable(cmap="icefire", norm=norm)
sm.set_array([])  # required for colorbar

cbar_ax = fig.add_axes([0.25, -0.06, 0.5, 0.02])  # [left, bottom, width, height]
cbar = fig.colorbar(sm, cax=cbar_ax, orientation="horizontal")

# Add centered ticks (e.g., 5 evenly spaced ticks)
ticks = np.linspace(global_min, global_max, 5) 
cbar.set_ticks(ticks) 
cbar.ax.set_xticklabels([f"{t:.2f}" for t in ticks]) # format labels cbar.set_label("Mode magnitude")
cbar.set_label("Mode magnitude")

plt.suptitle(f"DMD Spatial Modes \n(Top {topk}, $Re$ = {Re_target})", fontsize=16, y=0.95)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


In [ ]:
# Plot modal dynamics comparison true vs DMD for training parameter i.e., Reynolds number

Re_target = 130
dmd_model = dmd_dict[Re_target]

# DMD dynamics B ∈ ℂ^(r × n_time)
coeffs_dmd = dmd_model.dynamics.detach().cpu().numpy()

# Time vector truncated to match dynamics length
times_num = np.array(sampled_times_dict[Re_target], dtype=float)[:coeffs_dmd.shape[1]]
dt = times_num[1] - times_num[0]

# True snapshots projected into DMD basis
X_true = snapshot_dict[Re_target]
modes = dmd_model.modes.detach().cpu().numpy()
coeffs_true_in_dmd_basis = np.linalg.pinv(modes) @ X_true
coeffs_true_truncated = coeffs_true_in_dmd_basis[:, :coeffs_dmd.shape[1]]

# Select top-k modes
topk = 4
modes_to_plot = dmd_model.top_modes(n=topk, integral=False).cpu().numpy()

# Modal coefficient comparison (time domain)
fig, axes = plt.subplots(topk, 1,
                         figsize=(12, 3 * topk),
                         sharex=True)

for i, mode in enumerate(modes_to_plot):
    # Clean arrays
    modal_true_clean = np.asarray(coeffs_true_truncated[mode], dtype=np.float64)
    modal_dmd_clean  = np.asarray(coeffs_dmd[mode], dtype=np.float64)

    # Plot time series of modal coefficients
    ax = axes[i]
    line1, = ax.plot(times_num, modal_true_clean, color="black", lw=1.5)
    line2, = ax.plot(times_num, modal_dmd_clean, color="red", linestyle="--", lw=1.5)

    # Labels and titles
    ax.set_ylabel("Coefficient Value", fontsize=12)
    ax.set_title(f"Mode $\\Phi_{{{mode}}}$", fontsize=12, pad=6)
    ax.grid(True, alpha=0.6)

    # Stacked legend
    ax.legend([line1, line2], ["True", "ParametricDMD"], fontsize=11)

# Shared labels and layout
fig.align_ylabels(axes)
axes[-1].set_xlabel("$t$ (Time in seconds)", fontsize=14)
plt.suptitle(f"Modal Coefficient Comparison — True vs DMD \n(Top {topk} modes, Re={Re_target})",
             fontsize=16, y=0.97)
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()


In [ ]:
# Plot frequency spectrum of modal coefficients true vs DMD for training parameter i.e., Reynolds number

Re_target = 130
dmd_model = dmd_dict[Re_target]

# DMD dynamics B ∈ ℂ^(r × n_time)
coeffs_dmd = dmd_model.dynamics.detach().cpu().numpy()

# Time vector truncated to match dynamics length
times_num = np.array(sampled_times_dict[Re_target], dtype=float)[:coeffs_dmd.shape[1]]
dt = times_num[1] - times_num[0]

# True snapshots projected into DMD basis
X_true = snapshot_dict[Re_target]
modes = dmd_model.modes.detach().cpu().numpy()
coeffs_true_in_dmd_basis = np.linalg.pinv(modes) @ X_true
coeffs_true_truncated = coeffs_true_in_dmd_basis[:, :coeffs_dmd.shape[1]]

# Select top-k modes
topk = 4
modes_to_plot = dmd_model.top_modes(n=topk, integral=False).cpu().numpy()

# FFT comparison of True vs ParametricDMD modal dynamics
fig, axes = plt.subplots(topk, 1,
                         figsize=(12, 3 * topk),
                         sharex=True)

for i, mode in enumerate(modes_to_plot):
    # Clean arrays
    modal_true_clean = np.asarray(coeffs_true_truncated[mode], dtype=np.float64)
    modal_dmd_clean  = np.asarray(coeffs_dmd[mode], dtype=np.float64)

    # FFTs
    fft_true = np.abs(np.fft.rfft(modal_true_clean))
    fft_dmd  = np.abs(np.fft.rfft(modal_dmd_clean))
    freqs    = np.fft.rfftfreq(modal_dmd_clean.shape[0], d=dt)

    # Plot
    ax = axes[i]
    line1, = ax.plot(freqs, fft_true, color="black", lw=1.5)
    line2, = ax.plot(freqs, fft_dmd, color="red", linestyle="--", lw=1.5)

    # Labels and titles
    ax.set_ylabel("Spectral Amplitude", fontsize=12)
    ax.set_title(f"Mode $\\Phi_{{{mode}}}$", fontsize=12, pad=6)
    ax.grid(True, alpha=0.6)

    # Stacked legend
    ax.legend([line1, line2], ["True", "ParametricDMD"], fontsize=11)

# Shared labels and layout
fig.align_ylabels(axes)
axes[-1].set_xlabel("Frequency (Hz)", fontsize=14)
plt.suptitle(f"Frequency Spectrum of Modal Coefficients — True vs DMD \n(Top {topk} modes, Re={Re_target})",
             fontsize=16, y=0.97)
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()


In [ ]:
# Plot flow comparison true vs DMD for training parameter i.e., Reynolds number

# Target Reynolds number
Re_target = 130
t_start, t_end = 12.0, 13.0
granularity = 10

# DMD and true data
X_forecast = dmd_dict[Re_target].reconstruction.detach().cpu().numpy()
t_forecast = np.array(sampled_times_dict[Re_target], dtype=float)
X_true     = snapshot_dict[Re_target]

# Spatial coordinates
coords     = masked_coords_dict[Re_target]
x_flat, y_flat = coords[:, 0], coords[:, 1]
num_points = num_points_dict[Re_target]
tri        = mtri.Triangulation(x_flat, y_flat)

# Select indices
mask     = (t_forecast >= t_start) & (t_forecast <= t_end)
indices  = np.where(mask)[0][::granularity]

# Precompute magnitudes
true_mags, fore_mags, resid_mags = [], [], []
for i in indices:
    ux_true, uy_true = X_true[:num_points, i], X_true[num_points:, i]
    ux_fore, uy_fore = X_forecast[:num_points, i], X_forecast[num_points:, i]
    mag_true  = np.sqrt(ux_true**2 + uy_true**2)
    mag_fore  = np.sqrt(ux_fore**2 + uy_fore**2)
    mag_resid = mag_true - mag_fore
    true_mags.append(mag_true)
    fore_mags.append(mag_fore)
    resid_mags.append(mag_resid)

# Global color limits
vmin = min(m.min() for m in true_mags + fore_mags)
vmax = max(m.max() for m in true_mags + fore_mags)

# Shared normalization for True/Forecast
norm_main = Normalize(vmin=vmin, vmax=vmax)
sm_shared = ScalarMappable(norm=norm_main, cmap="icefire")
sm_shared.set_array([])
fixed_ticks_main = np.linspace(vmin, vmax, 10)

# Residual scaling
resid_min = min(m.min() for m in resid_mags)
resid_max = max(m.max() for m in resid_mags)
fixed_ticks_resid = np.linspace(resid_min, resid_max, 10)
sm_resid = ScalarMappable(norm=Normalize(vmin=resid_min, vmax=resid_max), cmap="icefire")
sm_resid.set_array([])

# Plot
num_rows = len(indices)
fig, axes = plt.subplots(num_rows, 3, figsize=(14, 3 * num_rows))
fig.suptitle(f"Flow Comparison — True vs DMD \n($Re$ = {Re_target})", fontsize=18, y=0.94)

if num_rows == 1:
    axes = np.expand_dims(axes, axis=0)

for row, i in enumerate(indices):
    t_val = t_forecast[i]
    mags  = [true_mags[row], fore_mags[row], resid_mags[row]]

    # True plot
    ax_true = axes[row, 0]
    ax_true.tricontourf(tri, mags[0], levels=200, cmap="icefire", vmin=vmin, vmax=vmax)
    ax_true.add_patch(Circle((0.2, 0.2), 0.05, color="black", zorder=10))
    ax_true.set_aspect("equal"); ax_true.set_xticks([]); ax_true.set_yticks([])
    for spine in ax_true.spines.values(): spine.set_visible(False)
    fig.colorbar(sm_shared, ax=ax_true, shrink=0.75, pad=0.02, ticks=fixed_ticks_main)
    ax_true.set_ylabel(f"$t$ = {t_val:.2f} s", fontsize=15, rotation=90, labelpad=25,
                       bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=2))

    # Forecast plot
    ax_fore = axes[row, 1]
    ax_fore.tricontourf(tri, mags[1], levels=200, cmap="icefire", vmin=vmin, vmax=vmax)
    ax_fore.add_patch(Circle((0.2, 0.2), 0.05, color="black", zorder=10))
    ax_fore.set_aspect("equal"); ax_fore.set_xticks([]); ax_fore.set_yticks([])
    for spine in ax_fore.spines.values(): spine.set_visible(False)
    fig.colorbar(sm_shared, ax=ax_fore, shrink=0.75, pad=0.02, ticks=fixed_ticks_main)

    # Residual plot
    ax_res = axes[row, 2]
    ax_res.tricontourf(tri, mags[2], levels=200, cmap="icefire", vmin=resid_min, vmax=resid_max)
    ax_res.add_patch(Circle((0.2, 0.2), 0.05, color="black", zorder=10))
    ax_res.set_aspect("equal"); ax_res.set_xticks([]); ax_res.set_yticks([])
    for spine in ax_res.spines.values(): spine.set_visible(False)
    fig.colorbar(sm_resid, ax=ax_res, shrink=0.75, pad=0.02, ticks=fixed_ticks_resid)

# Column titles only once
axes[0, 0].set_title("True Magnitude", fontsize=15, pad=12,
                     bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=2))
axes[0, 1].set_title("DMD Reconstruction", fontsize=15, pad=12,
                     bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=2))
axes[0, 2].set_title(r"Residual $(U_{\mathrm{True}} - U_{\mathrm{DMD}})$", fontsize=15, pad=12,
                     bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=2))

plt.tight_layout(rect=[0, 0.03, 1, 0.95], h_pad=1)
plt.show()


In [ ]:
# Plot DMD reconstruction error for training parameter i.e., Reynolds number

# Target Reynolds number
Re = 130

# Snapshot matrices
# True snapshots: X_true ∈ ℝ^(2·n_points × n_time)
X_true = snapshot_dict[Re]
# DMD reconstruction: X_dmd ∈ ℝ^(2·n_points × n_time)
X_dmd  = dmd_dict[Re].reconstruction.detach().cpu().numpy()

# Time vector t ∈ ℝ^(n_time)
time_vec = np.array(sampled_times_dict[Re], dtype=float)

# Compute reconstruction error over time
# Absolute L2 error: ||U_true - U_dmd||_2
abs_error = np.linalg.norm(X_true - X_dmd, axis=0)

# Relative error: ||U_true - U_dmd||_2 / ||U_true||_2
rel_error = abs_error / np.linalg.norm(X_true, axis=0)

# Plotting
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
ax.plot(time_vec, rel_error, label="Relative L2 Error", color="red")
#ax.plot(time_vec, abs_error, label="Absolute $L^2$ error", color="black")

ax.set_xlabel("$t$ (Time in seconds)")
ax.set_ylabel("Error")
ax.set_title(f"DMD Reconstruction Error \n(Re = {Re})")
ax.grid(True)
ax.legend()
plt.show()


## Step 3. Forecasting

 For each training parameter $\mu_j \in P = \{\mu_{\scriptstyle 1}, \mu_{\scriptstyle 2}, \dots, \mu_p\}
$ i.e., Reynolds numbers $Re$, the forecast snapshots at unseen time instants are $X_{\text{forecast}}^{(\mu_j)} \in \mathbb{R}^{n_{\text{space}} \times n_{\text{forecast}}}$
 $$X_{\text{forecast}}^{(\mu_j)}(t^\star) \;\; \text{with } t^\star \in \{t_1, t_2, \dots, t_{n_{\text{forecast}}}\}.
 $$ 

where  
- $\mu_j$ : $j$‑th training parameter in the set $P$  
- $n_{\text{space}}$ : number of spatial degrees of freedom  
- $n_{\text{forecast}}$ : number of forecast time instants  
- $t^\star$ : a specific forecast instant chosen from the forecast window (e.g., for comparison with true data)


In [ ]:
forecast_dict = {}

# Forecast horizon
n_steps = 500
dt = 0.01

for Re, dmd_model in dmd_dict.items():
    # Initial condition = first snapshot column
    x0 = pt.tensor(snapshot_dict[Re][:, 0], dtype=pt.float32)

    # Forecast trajectory: X̂^(Re)(t) ∈ ℝ^(space_dim × (n_steps+1))
    X_forecast = dmd_model.predict(x0, n_steps=n_steps)

    # Physical forecast start time = last training snapshot time
    t0_forecast = float(sampled_times_dict[Re][-1])

    # Spectral data from DMD block
    Phi = dmd_model.modes.detach().cpu().numpy()       # shape (n_space, r)
    lam = dmd_model.eigvals.detach().cpu().numpy()     # discrete-time eigenvalues
    s   = dmd_model.eigvals_cont.detach().cpu().numpy()  # continuous-time eigenvalues

    # Raw forecast coefficients B_raw^(Re) = Φ^† X_fore
    Phi_pinv = np.linalg.pinv(Phi)
    B_raw = Phi_pinv @ X_forecast.detach().cpu().numpy()   # shape (r, n_forecast)

    # Importance index I_j = ||φ_j||_1 * Σ_k |b_jk|
    mode_norms    = np.linalg.norm(Phi, ord=1, axis=0)
    coeff_abs_sum = np.sum(np.abs(B_raw), axis=1)
    I = mode_norms * coeff_abs_sum

    # Sort modes
    sort_idx   = dmd_model.top_modes(n=len(lam))
    Phi_sorted = Phi[:, sort_idx]
    s_sorted   = s[sort_idx]
    B_forecast = B_raw[sort_idx, :]   # aligned forecast coefficients

    # Store everything in forecast_dict
    forecast_dict[Re] = {
        "t_forecast": t0_forecast + np.arange(n_steps + 1) * dt,
        "X_forecast": X_forecast.detach().cpu().numpy(),
        "Phi_forecast": Phi,
        "lam_forecast": lam,
        "s_forecast": s,
        "B_raw": B_raw,
        "B_forecast": B_forecast,
        "importance": I,
        "Phi_sorted": Phi_sorted,
        "s_sorted": s_sorted
    }

    print(f"Re = {Re}: forecasted trajectory shape = {forecast_dict[Re]['X_forecast'].shape}")


In [ ]:
# Plot modal dynamics true vs forecasted DMD for training parameter i.e., Reynolds number

Re_target = 160
dmd_model = dmd_dict[Re_target]

# Forecast results from your forecast_dict
X_forecast = forecast_dict[Re_target]["X_forecast"]
t_forecast = forecast_dict[Re_target]["t_forecast"]

# True snapshots
X_true = snapshot_dict[Re_target]

# DMD spatial modes Φ ∈ ℂ^(n_space × r)
modes = dmd_model.modes.detach().cpu().numpy()

# Project true snapshots into DMD basis
coeffs_true_in_dmd_basis = np.linalg.pinv(modes) @ X_true

# Truncate to forecast horizon length
coeffs_true_truncated = coeffs_true_in_dmd_basis[:, :X_forecast.shape[1]]

# Project forecasted snapshots into DMD basis
coeffs_forecast_in_dmd_basis = np.linalg.pinv(modes) @ X_forecast

# Select top-k modes
topk = 6
modes_to_plot = dmd_model.top_modes(n=topk, integral=True).cpu().numpy()

fig, axes = plt.subplots(topk, 1,
                         figsize=(12, 3 * topk),
                         sharex=True)

for idx, mode in enumerate(modes_to_plot):
    ax = axes[idx]
    ax.plot(t_forecast, coeffs_true_truncated[mode].real,
            label="True", color='black', lw=1.5)
    ax.plot(t_forecast, coeffs_forecast_in_dmd_basis[mode].real,
            label="Forecasted DMD", linestyle='--', color='red', lw=1.5)
    ax.set_ylabel("Amplitude", fontsize=14)
    ax.set_title(f"Mode $\\Phi_{{{mode}}}$", fontsize=14, pad=6)
    ax.grid(True, alpha=0.6)
    ax.legend(fontsize=13)

fig.align_ylabels(axes)
axes[-1].set_xlabel("$t$ (Time in seconds)", fontsize=15)
plt.suptitle(f"Forecasted vs True Modal Dynamics \n(Top {topk} modes, Re={Re_target})",
             fontsize=18, y=0.97)
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()


In [ ]:
# Plot Frequency spectrum of modal dynamics true vs forecasted DMD for training parameter i.e., Reynolds number
Re_target = 160

# Forecast results from your forecast_dict
X_forecast = forecast_dict[Re_target]["X_forecast"]
t_forecast = forecast_dict[Re_target]["t_forecast"]
dt = t_forecast[1] - t_forecast[0]

# True snapshots
X_true = snapshot_dict[Re_target]

# DMD spatial modes Φ ∈ ℂ^(n_space × r)
modes = dmd_model.modes.detach().cpu().numpy()

# Project true snapshots into DMD basis
coeffs_true_in_dmd_basis = np.linalg.pinv(modes) @ X_true
coeffs_true_truncated = coeffs_true_in_dmd_basis[:, :X_forecast.shape[1]]

# Project forecasted snapshots into DMD basis
coeffs_forecast_in_dmd_basis = np.linalg.pinv(modes) @ X_forecast

# Select top-k modes
topk = 4
modes_to_plot = dmd_model.top_modes(n=topk, integral=False).cpu().numpy()

# FFT comparison of True vs Forecasted modal dynamics
fig, axes = plt.subplots(topk, 1,
                         figsize=(12, 3 * topk),
                         sharex=True)

for i, mode in enumerate(modes_to_plot):
    # Clean arrays
    modal_true_clean     = np.asarray(coeffs_true_truncated[mode], dtype=np.float64)
    modal_forecast_clean = np.asarray(coeffs_forecast_in_dmd_basis[mode], dtype=np.float64)

    # FFTs
    fft_true     = np.abs(np.fft.rfft(modal_true_clean))
    fft_forecast = np.abs(np.fft.rfft(modal_forecast_clean))
    freqs        = np.fft.rfftfreq(modal_forecast_clean.shape[0], d=dt)

    # Plot
    ax = axes[i]
    line1, = ax.plot(freqs, fft_true, color="black", lw=1.5)
    line2, = ax.plot(freqs, fft_forecast, color="red", linestyle="--", lw=1.5)

    # Labels and titles
    ax.set_ylabel("Spectral Amplitude", fontsize=12)
    ax.set_title(f"Mode $\\Phi_{{{mode}}}$", fontsize=12, pad=6)
    ax.grid(True, alpha=0.6)

    # Stacked legend
    ax.legend([line1, line2], ["True", "Forecasted DMD"], fontsize=11)

# Shared labels and layout
fig.align_ylabels(axes)
axes[-1].set_xlabel("Frequency (Hz)", fontsize=14)
plt.suptitle(f"Frequency Spectrum of Modal Coefficients — True vs Forecasted DMD \n(Top {topk} modes, Re={Re_target})",
             fontsize=16, y=0.97)
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()



In [ ]:
# Plot flow comparison: true vs forecasted DMD for training parameter i.e., Reynolds number

Re_target = 120
t_start, t_end = 15.0, 16.0
granularity = 10

# Forecast and true data
X_forecast = forecast_dict[Re_target]["X_forecast"]
t_forecast = forecast_dict[Re_target]["t_forecast"]
X_true = snapshot_dict[Re_target]

# Spatial coordinates
coords = masked_coords_dict[Re_target]
x_flat, y_flat = coords[:, 0], coords[:, 1]
num_points = num_points_dict[Re_target]
tri = mtri.Triangulation(x_flat, y_flat)

# Select indices
mask = (t_forecast >= t_start) & (t_forecast <= t_end)
indices = np.where(mask)[0][::granularity]

# Precompute magnitudes
true_mags, fore_mags, resid_mags = [], [], []
for i in indices:
    ux_true, uy_true = X_true[:num_points, i], X_true[num_points:, i]
    ux_fore, uy_fore = X_forecast[:num_points, i], X_forecast[num_points:, i]
    mag_true = np.sqrt(ux_true**2 + uy_true**2)
    mag_fore = np.sqrt(ux_fore**2 + uy_fore**2)
    mag_resid = mag_true - mag_fore
    true_mags.append(mag_true)
    fore_mags.append(mag_fore)
    resid_mags.append(mag_resid)

# Global color limits
vmin = min(m.min() for m in true_mags + fore_mags)
vmax = max(m.max() for m in true_mags + fore_mags)

# Shared normalization for True/Forecast
norm_main = Normalize(vmin=vmin, vmax=vmax)
sm_shared = ScalarMappable(norm=norm_main, cmap="icefire")
sm_shared.set_array([])
fixed_ticks_main = np.linspace(vmin, vmax, 10)

# Residual scaling
resid_min = min(m.min() for m in resid_mags)
resid_max = max(m.max() for m in resid_mags)
fixed_ticks_resid = np.linspace(resid_min, resid_max, 10)
sm_resid = ScalarMappable(norm=Normalize(vmin=resid_min, vmax=resid_max), cmap="icefire")
sm_resid.set_array([])

# Plot
num_rows = len(indices)
fig, axes = plt.subplots(num_rows, 3, figsize=(14, 3 * num_rows))
fig.suptitle(f"Flow Comparison — True vs Forecasted DMD\n($Re$ = {Re_target})", fontsize=18, y=0.94)

if num_rows == 1:
    axes = np.expand_dims(axes, axis=0)

for row, i in enumerate(indices):
    t_val = t_forecast[i]
    mags = [true_mags[row], fore_mags[row], resid_mags[row]]

    # True plot
    ax_true = axes[row, 0]
    ax_true.tricontourf(tri, mags[0], levels=200, cmap="icefire", vmin=vmin, vmax=vmax)
    ax_true.add_patch(Circle((0.2, 0.2), 0.05, color="black", zorder=10))
    ax_true.set_aspect("equal"); ax_true.set_xticks([]); ax_true.set_yticks([])
    for spine in ax_true.spines.values(): spine.set_visible(False)
    fig.colorbar(sm_shared, ax=ax_true, shrink=0.75, pad=0.02, ticks=fixed_ticks_main)
    ax_true.set_ylabel(f"$t$ = {t_val:.2f} s", fontsize=15, rotation=90, labelpad=25,
                       bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=2))

    # Forecast plot
    ax_fore = axes[row, 1]
    ax_fore.tricontourf(tri, mags[1], levels=200, cmap="icefire", vmin=vmin, vmax=vmax)
    ax_fore.add_patch(Circle((0.2, 0.2), 0.05, color="black", zorder=10))
    ax_fore.set_aspect("equal"); ax_fore.set_xticks([]); ax_fore.set_yticks([])
    for spine in ax_fore.spines.values(): spine.set_visible(False)
    fig.colorbar(sm_shared, ax=ax_fore, shrink=0.75, pad=0.02, ticks=fixed_ticks_main)

    # Residual plot
    ax_res = axes[row, 2]
    ax_res.tricontourf(tri, mags[2], levels=200, cmap="icefire", vmin=resid_min, vmax=resid_max)
    ax_res.add_patch(Circle((0.2, 0.2), 0.05, color="black", zorder=10))
    ax_res.set_aspect("equal"); ax_res.set_xticks([]); ax_res.set_yticks([])
    for spine in ax_res.spines.values(): spine.set_visible(False)
    fig.colorbar(sm_resid, ax=ax_res, shrink=0.75, pad=0.02, ticks=fixed_ticks_resid)

# Column titles only once
axes[0, 0].set_title("True Magnitude", fontsize=15, pad=12,
                     bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=2))
axes[0, 1].set_title("Forecasted DMD Reconstruction", fontsize=15, pad=12,
                     bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=2))
axes[0, 2].set_title(r"Residual $(U_{\mathrm{True}} - U_{\mathrm{Forecasted DMD}})$", fontsize=15, pad=12,
                     bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=2))

plt.tight_layout(rect=[0, 0.03, 1, 0.95], h_pad=1)
plt.show()


In [ ]:
# Plot forecasted DMD reconstruction error for training parameter i.e., Reynolds number

Re_target = 120

# Forecast results
X_forecast = forecast_dict[Re_target]["X_forecast"]
t_forecast = forecast_dict[Re_target]["t_forecast"]

# True snapshots (truncate to forecast horizon length)
X_true = snapshot_dict[Re_target][:, :X_forecast.shape[1]]

# Compute error norms
abs_error = np.linalg.norm(X_true - X_forecast, axis=0)
rel_error = abs_error / np.linalg.norm(X_true, axis=0)

# Plot error vs time
fig, ax = plt.subplots(figsize=(8,4))
ax.plot(t_forecast, rel_error, 'r-', lw=2, label="Relative $L^2$ error")
#ax.plot(t_forecast, abs_error, 'k--', lw=1.5, label="Absolute $L^2$ error")
ax.set_xlabel("$t$ (Time in seconds)")
ax.set_ylabel("Error")
ax.set_title(f"Forecasted DMD Reconstruction Error \n(Re={Re_target})")
ax.grid(True, alpha=0.6)
ax.legend()
plt.tight_layout()
plt.show()


## Step 4. Interpolation 

## Step 4.1 Mode importance sorting  
Rank modes by importance using their spatial norm and amplitude magnitude.

$$
\Large I_j^{(\mu_j)} = \| \phi_j^{(\mu_j)} \|_1 \cdot \sum_k \big| b_{jk}^{(\mu_j)} \big|
$$

- $I_j^{(\mu_j)}$: importance index of mode $j$  
- $\phi_j^{(\mu_j)}$: $j$‑th DMD mode vector  
- $\| \cdot \|_1$: L1 norm (sum of absolute values)  
- $b_{jk}^{(\mu_j)}$: amplitude of mode $j$ at time $k$ 


In [26]:
selected_modes_dict   = {}
selected_eigvals_dict = {}
importance_dict       = {}

dt = 0.01  # forecast snapshot spacing

# choose a reference case for alignment (closest to target Re)
Re_ref  = min(Re_list, key=lambda R: abs(R - Re_target))  # automatic nearest neighbor
Phi_ref = forecast_dict[Re_ref]["Phi_forecast"]           # keep full rank

for Re in Re_list:
    # Forecast snapshots X_fore^(Re) ∈ ℝ^(n_space × n_forecast)
    X_fore = forecast_dict[Re]["X_forecast"]

    # Φ^(Re): forecast-based DMD modes
    Phi = forecast_dict[Re]["Phi_forecast"]

    # λ^(Re): discrete-time eigenvalues
    lam = forecast_dict[Re]["lam_forecast"]

    # s^(Re): continuous-time eigenvalues
    s = dmd_dict[Re].eigvals_cont.detach().cpu().numpy()

    # B_raw^(Re): forecast mode coefficients = Φ^† X_fore
    Phi_pinv = np.linalg.pinv(Phi)
    B_raw = Phi_pinv @ X_fore

    # Importance index I_j = ||φ_j||_1 * Σ_k |b_jk|
    mode_norms    = np.linalg.norm(Phi, ord=1, axis=0)
    coeff_abs_sum = np.sum(np.abs(B_raw), axis=1)
    I = mode_norms * coeff_abs_sum
    importance_dict[Re] = I

    # Sort modes by descending importance
    sort_idx   = np.argsort(-I)
    Phi_sorted = Phi[:, sort_idx]
    s_sorted   = s[sort_idx]
    B_sorted   = B_raw[sort_idx, :]

    # Conjugate pair ordering
    ordered_modes, ordered_eigs, ordered_coeffs = [], [], []
    used = np.zeros(len(s_sorted), dtype=bool)
    for j in range(len(s_sorted)):
        if used[j]:
            continue
        eig = s_sorted[j]
        conj_idx = np.where(np.isclose(s_sorted, np.conj(eig)))[0]
        if len(conj_idx) > 0 and conj_idx[0] != j:
            k = conj_idx[0]
            used[j] = True; used[k] = True
            if np.imag(eig) >= 0:
                ordered_eigs.extend([eig, s_sorted[k]])
                ordered_modes.extend([Phi_sorted[:, j], Phi_sorted[:, k]])
                ordered_coeffs.extend([B_sorted[j, :], B_sorted[k, :]])
            else:
                ordered_eigs.extend([s_sorted[k], eig])
                ordered_modes.extend([Phi_sorted[:, k], Phi_sorted[:, j]])
                ordered_coeffs.extend([B_sorted[k, :], B_sorted[j, :]])
        else:
            used[j] = True
            ordered_eigs.append(eig)
            ordered_modes.append(Phi_sorted[:, j])
            ordered_coeffs.append(B_sorted[j, :])

    # Keep ALL modes (no truncation)
    ordered_modes  = np.column_stack(ordered_modes)
    ordered_eigs   = np.array(ordered_eigs)
    ordered_coeffs = np.vstack(ordered_coeffs)

    # MRPWI alignment relative to reference case
    aligned_modes, aligned_coeffs = [], []
    for k in range(ordered_modes.shape[1]):
        phi_ref = Phi_ref[:, k]
        phi_cur = ordered_modes[:, k]
        angle   = np.angle(np.vdot(phi_ref, phi_cur))
        aligned_modes.append(phi_cur * np.exp(-1j * angle))
        aligned_coeffs.append(ordered_coeffs[k, :] * np.exp(-1j * angle))

    aligned_modes  = np.column_stack(aligned_modes)
    aligned_coeffs = np.vstack(aligned_coeffs)

    # Store results
    selected_modes_dict[Re]   = aligned_modes
    selected_eigvals_dict[Re] = ordered_eigs
    forecast_dict[Re]["B_raw"]      = B_raw
    forecast_dict[Re]["B_forecast"] = aligned_coeffs


## Step 4.2. Interpolation Stencil Selection  

Choose the nearest training Reynolds numbers around the unseen parameter $\mu^*$ to form interpolation stencils. Neighbor selection is performed using the **Grassmann geodesic distance** between subspaces spanned by the DMD modes, ensuring geometric consistency (Edelman et al., 1998; Giovanis & Shields, 2019).

$$
(\mu_{j1}, \mu_{j2}, \dots, \mu_{jN_p})
$$

- $\mu_{ji}$ : the $i$‑th neighbor Reynolds number chosen from the training set, selected by minimizing the Grassmann geodesic distance  
- $N_p$ : stencil size (number of neighbors used for interpolation)  
- $\mu^*$ : target Reynolds number where we want to forecast  
 


In [ ]:

# Step 4.2. Interpolation Stencil Selection using Grassmann geodesic distance
# References: Edelman et al. (1998); Giovanis & Shields (2019)

Re_list   = sorted(selected_modes_dict.keys())   # available training Reynolds numbers {μ_j}
Re_test   = 190                                             # target parameter μ*

stencil_sets = {}

for Np in [2, 4, 6]:
    # Ensure even stencil size
    if Np % 2 == 1:
        Np = Np - 1

    # Sort cases by Grassmann distance to target parameter μ*
    # Note: since μ* is unseen, we compare training subspaces to each other
    sorted_neighbors = sorted(
        Re_list,
        key=lambda Re: abs(Re - Re_test)  # parameter proximity for unseen μ*
    )

    # Select Np closest cases {μ_j1, …, μ_jNp}
    stencil = sorted_neighbors[:Np]

    # Re‑index sequentially and sort ascending {μ_j}
    stencil = sorted(stencil)

    # Store stencil set for later interpolation of Φ^(μ) and S^(μ)
    stencil_sets[Np] = stencil

print("Selected stencils:", stencil_sets)

# Step 2: for each stencil, choose reference case = closest in Grassmann metric to others
for Np, pts in stencil_sets.items():
    Re_ref = min(
        pts,
        key=lambda Re: np.linalg.norm(
            np.arccos(
                np.clip(
                    np.linalg.svd(
                        np.linalg.qr(selected_modes_dict[Re])[0].conj().T @ 
                        np.linalg.qr(selected_modes_dict[pts[0]])[0],
                        compute_uv=False
                    ),
                    -1.0, 1.0
                )
            )
        )
    )
    Phi_ref = selected_modes_dict[Re_ref]
    # use Phi_ref for alignment in this stencil


## Step 4.3. Modal interpolation (MRPWI)  
Realign the DMD modes across parameters, then interpolate them pointwise using Lagrange weights.

$$
\phi_j^{(μ^*)} = \sum_{i=1}^{N_p} L_i(μ^*) \, \phi_j^{(μ_{ji})}
$$

- $\phi_j^{(μ^*)}$ : interpolated $j$‑th mode at target Reynolds number $μ$*  
- $L_i(μ^*)$ : Lagrange interpolation weight for neighbor $i$, evaluated at $μ$*  
- $\phi_j^{(μ_{ji})}$ : $j$‑th mode at neighbor Reynolds number $μ_{ji}$  
- $N_p$ : stencil size (number of neighbors used)  



In [ ]:
# Step 4.3. Lagrange Interpolation of Re-aligned DMD Modes Φ^(μ*) at test parameter μ*

Re_test = 190 
Nn = selected_modes_dict[Re_list[0]].shape[0]   # spatial dimension n_space
r  = selected_modes_dict[Re_list[0]].shape[1]   # number of retained modes r

# Dictionaries to store interpolated modes
interp_modes_aligned   = {}   # Φ^(μ*) interpolated aligned DMD modes at target parameter
interp_modes_unaligned = {}   # Φ^(μ*) interpolated raw DMD modes at target parameter

for Np, pts in stencil_sets.items():
    # Reference case for alignment: Φ^(Re_ref)
    Re_ref  = min(pts, key=lambda Re: abs(Re - Re_test))
    Phi_ref = selected_modes_dict[Re_ref]       # Φ^(Re_ref)

    # Interpolation of aligned DMD modes Φ^(μ)
    Phi_interp_aligned = np.zeros((Nn, r), dtype=complex)
    for j in range(r):                          # loop over retained modes φ_j
        for k in range(Nn):                     # spatial index
            phi_k_values = [selected_modes_dict[Re][k, j] for Re in pts]  # φ_j^(Re)
            result = 0.0
            for i in range(Np):
                L_i = 1.0                       # Lagrange basis polynomial L_i(μ*)
                for m in range(Np):
                    if m != i:
                        L_i *= (Re_test - pts[m]) / (pts[i] - pts[m])
                result += phi_k_values[i] * L_i
            Phi_interp_aligned[k, j] = result   # φ_j^(μ*)
    # Apply phase alignment relative to reference Φ^(Re_ref)
    aligned_modes = []
    for j in range(r):
        angle = np.angle(np.vdot(Phi_ref[:, j], Phi_interp_aligned[:, j]))
        aligned_modes.append(Phi_interp_aligned[:, j] * np.exp(-1j * angle))  # φ_j^(μ*) aligned
    Phi_interp_aligned = np.column_stack(aligned_modes)
    interp_modes_aligned[Np] = Phi_interp_aligned   # Φ^(μ*) aligned

    # Interpolation of unaligned raw DMD modes Φ^(μ) from forecast modes
    Phi_interp_unaligned = np.zeros((Nn, r), dtype=complex)
    for j in range(r):                          # φ_j^(Re) raw
        for k in range(Nn):
            phi_k_values = [dmd_dict[Re].modes.detach().cpu().numpy()[k, j] for Re in pts]  # φ_j^(Re)
            result = 0.0
            for i in range(Np):
                L_i = 1.0                       # Lagrange basis polynomial L_i(μ*)
                for m in range(Np):
                    if m != i:
                        L_i *= (Re_test - pts[m]) / (pts[i] - pts[m])
                result += phi_k_values[i] * L_i
            Phi_interp_unaligned[k, j] = result # φ_j^(μ*)
    interp_modes_unaligned[Np] = Phi_interp_unaligned   # Φ^(μ*) unaligned


## Step 4.4. Eigenvalue interpolation (pointwise)  
Interpolate the continuous‑time eigenvalues across the stencil using Lagrange weights.

$$
s_j^{(\mu^*)} = \sum_{i=1}^{N_p} L_i(\mu^*) \, s_j^{(\mu_{ji})}
$$

- $s_j^{(\mu^*)}$ : interpolated $j$‑th continuous eigenvalue at test Reynolds number $μ*$  
- $L_i(\mu^*)$ : Lagrange interpolation weight for neighbor $i$, evaluated at $μ^*$ 
- $s_j^{(\mu_{ji})}$ : $j$‑th continuous eigenvalue at neighbouring Reynolds number $μ_{ji}$
- $N_p$ : stencil size (number of neighbors used) 


In [ ]:
# Step 4.4. Lagrange Interpolation of DMD Eigenvalues s^(μ*) at test parameter μ*

# Dictionaries to store interpolated eigenvalues
interp_eigs_aligned   = {}   # s^(μ*)_aligned interpolated from aligned s^(Re)
interp_eigs_unaligned = {}   # s^(μ*)_unaligned interpolated from raw λ^(Re)

Re_test = 190 
r  = selected_eigvals_dict[Re_list[0]].shape[0]   # number of retained modes
dt = 0.01                                         # snapshot spacing (forecast horizon)

for Np, pts in stencil_sets.items():
    # Interpolation of aligned continuous-time eigenvalues s^(Re)
    s_interp_aligned = np.zeros(r, dtype=complex)
    for j in range(r):
        s_j_values = [selected_eigvals_dict[Re][j] for Re in pts]  # s^(Re)_aligned
        result = 0.0
        for i in range(Np):
            L_i = 1.0
            for k in range(Np):
                if k != i:
                    L_i *= (Re_test - pts[k]) / (pts[i] - pts[k])
            result += s_j_values[i] * L_i
        s_interp_aligned[j] = result
    interp_eigs_aligned[Np] = s_interp_aligned   # s^(μ*)_aligned

    # Interpolation of unaligned continuous-time eigenvalues s^(Re) = log(λ^(Re))/dt
    s_interp_unaligned = np.zeros(r, dtype=complex)
    for j in range(r):
        lam_values = [dmd_dict[Re].eigvals.detach().cpu().numpy()[j] for Re in pts]  # λ^(Re)_raw
        s_j_values = [np.log(lam) / dt for lam in lam_values]                        # s^(Re)_unaligned
        result = 0.0
        for i in range(Np):
            L_i = 1.0
            for k in range(Np):
                if k != i:
                    L_i *= (Re_test - pts[k]) / (pts[i] - pts[k])
            result += s_j_values[i] * L_i
        s_interp_unaligned[j] = result
    interp_eigs_unaligned[Np] = s_interp_unaligned   # s^(μ*)_unaligned


## Step 4.5. Amplitude estimation for $\mu^*$ by Projection

Instead of interpolating amplitudes directly across the stencil, we compute them at the target parameter $\mu^*$ (i.e., Reynolds number) by projecting the initial condition snapshot onto the interpolated modal basis:

$$
b^{(\mu^*)} = \phi^{(\mu^*)\dagger} \, x_0^{(\mu^*)}
$$

- $b^{(\mu^*)}$ : modal amplitudes consistent with the interpolated modes
- $\phi^{(\mu^*)}$ : interpolated modal basis at target Reynolds number  
- $x_0^{(\mu^*)}$ : initial snapshot of the test case at $\mu^*$  




In [ ]:
# Step 4.5. Initial Condition Projection to obtain DMD Amplitudes b^(μ*) at test parameter μ*

Re_test = 190 
r = selected_modes_dict[Re_list[0]].shape[1]   # number of retained modes

# Dictionaries to store interpolated amplitudes
interp_amps_aligned   = {}   # b^(μ*) aligned amplitudes from initial condition projection
interp_amps_unaligned = {}   # b^(μ*) unaligned amplitudes from initial condition projection

for Np, pts in stencil_sets.items():
    # Interpolated modes Φ^(μ*)
    Phi_interp_aligned   = interp_modes_aligned[Np]
    Phi_interp_unaligned = interp_modes_unaligned[Np]

    # Initial condition at target parameter (first snapshot of test case)
    x0_test = snapshot_test_future[:, 0]

    # Compute amplitudes by projection: b^(μ*) = Φ^(μ*)† x0^(μ*)
    b_interp_aligned   = np.linalg.pinv(Phi_interp_aligned)   @ x0_test
    b_interp_unaligned = np.linalg.pinv(Phi_interp_unaligned) @ x0_test

    # Store results
    interp_amps_aligned[Np]   = b_interp_aligned
    interp_amps_unaligned[Np] = b_interp_unaligned


## Step 4.6. Interpoled trajectory reconstruction

Reconstruct the interpolated trajectory for the target parameter $\mu^*$ as: 



$$
X_{\text{interp}}^{(\mu^*)}(t) = \phi^{(\mu^*)} 
\left( b^{(\mu^*)} \odot e^{s^{(\mu^*)} t} \right)
$$




where  
- $\phi^{(\mu^*)} \in \mathbb{C}^{n_{\text{space}} \times r}$ : interpolated DMD modes  
- $s^{(\mu^*)} \in \mathbb{C}^r$ : interpolated eigenvalues  
- $b^{(\mu^*)} \in \mathbb{C}^r$ : interpolated amplitudes  
- $t$ : forecast time instants  


In [ ]:

recon_results_aligned   = {}
recon_results_unaligned = {}

Re_test = 190 
t_vec   = sampled_times_test_future              # use actual forecast times for test case
n_space = forecast_dict[Re_list[0]]["X_forecast"].shape[0]
r       = selected_modes_dict[Re_list[0]].shape[1]

for Np in stencil_sets.keys():
    # Φ^(μ*): interpolated modes at target Reynolds
    Phi_aligned   = interp_modes_aligned[Np]     # Φ^(μ*)_aligned ∈ ℂ^(n_space × r)
    Phi_unaligned = interp_modes_unaligned[Np]   # Φ^(μ*)_unaligned ∈ ℂ^(n_space × r)

    # s^(μ*): interpolated continuous-time eigenvalues
    s_aligned   = interp_eigs_aligned[Np]        # s^(μ*)_aligned ∈ ℂ^r
    s_unaligned = interp_eigs_unaligned[Np]      # s^(μ*)_unaligned ∈ ℂ^r

    # b^(μ*): amplitudes from projection
    b_aligned   = interp_amps_aligned[Np]        # b^(μ*)_aligned ∈ ℂ^r
    b_unaligned = interp_amps_unaligned[Np]      # b^(μ*)_unaligned ∈ ℂ^r

    # Reconstruction X^(μ*)(t_k) = Φ^(μ*) diag(exp(s^(μ*) t_k)) b^(μ*)
    X_recon_aligned   = np.zeros((n_space, len(t_vec)), dtype=complex)
    X_recon_unaligned = np.zeros((n_space, len(t_vec)), dtype=complex)

    # Ensure time vector is numeric
    t_vec = np.array(sampled_times_test_future, dtype=float)
    t0    = t_vec[0]

    for k, t in enumerate(t_vec):
        # aligned reconstruction
        X_recon_aligned[:, k] = Phi_aligned @ (np.exp(s_aligned * (t - t0)) * b_aligned)

        # unaligned reconstruction
        X_recon_unaligned[:, k] = Phi_unaligned @ (np.exp(s_unaligned * (t - t0)) * b_unaligned)


    # Store reconstructions
    recon_results_aligned[Np]   = X_recon_aligned    # X^(μ*)_aligned ∈ ℂ^(n_space × n_time)
    recon_results_unaligned[Np] = X_recon_unaligned  # X^(μ*)_unaligned ∈ ℂ^(n_space × n_time)


In [ ]:
# Plot reconstruction errors vs stencil sizes for test parameter i.e., Reynolds number

errors_aligned   = {}
errors_unaligned = {}

for Np in stencil_sets.keys():
    X_true = snapshot_test_future   # ground truth snapshots
    X_pred_aligned   = recon_results_aligned[Np]
    X_pred_unaligned = recon_results_unaligned[Np]

    # RMSE relative error
    err_aligned = np.linalg.norm(X_pred_aligned - X_true) / np.linalg.norm(X_true)
    err_unaligned = np.linalg.norm(X_pred_unaligned - X_true) / np.linalg.norm(X_true)

    # Convert to percentage
    errors_aligned[Np]   = 100 * err_aligned
    errors_unaligned[Np] = 100 * err_unaligned

# Convert results to sorted lists for plotting
Np_values = sorted(stencil_sets.keys())
err_aligned_list   = [errors_aligned[Np]   for Np in Np_values]
err_unaligned_list = [errors_unaligned[Np] for Np in Np_values]

# Plot percentage error vs stencil size
fig, ax = plt.subplots(figsize=(7,5))
ax.plot(Np_values, err_aligned_list, 'o-', label="MRPWI Error (%)")
ax.plot(Np_values, err_unaligned_list, 's--', label="Unaligned PWI Error (%)")
ax.set_xlabel(r"Stencil size $N_p$")
ax.set_ylabel("Error (%)")
ax.set_title(f"Mode Re-aligned Pointwise Interpolation (MRPWI) Reconstruction Error \nRe={Re_test}", pad=10)
ax.legend()
ax.grid(True)

# Build table data
table_data = []
for Np, aligned, unaligned in zip(Np_values, err_aligned_list, err_unaligned_list):
    table_data.append([Np, f"{aligned:.2f}%", f"{unaligned:.2f}%"])

# Add table below plot
table = plt.table(cellText=table_data,
                  colLabels=["$N_p$", "MRPWI Error (%)", "Unaligned PWI Error (%)"],
                  loc="bottom",
                  cellLoc="center",
                  bbox=[0.0, -0.5, 1.0, 0.3])  # adjust vertical placement

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.2)

# Adjust layout to fit table
plt.subplots_adjust(left=0.1, bottom=0.3)

plt.show()


In [ ]:
# Plot bars for MRPWI reconstruction errors across stencil sizes for test parameter i.e., Reynolds number

# Prepare data
Np_values = sorted(stencil_sets.keys())
err_aligned_list   = [errors_aligned[Np]   for Np in Np_values]
err_unaligned_list = [errors_unaligned[Np] for Np in Np_values]


x = np.arange(len(Np_values))  # positions for bars
width = 0.35                   # bar width

# Create bar plot
plt.figure(figsize=(8,5))
plt.bar(x - width/2, err_aligned_list, width, label="MRPWI RMSE")
#plt.bar(x + width/2, err_unaligned_list, width, label="Unaligned PROM RMSE")

plt.xticks(x, Np_values)
plt.xlabel(r"Stencil size $N_p$")

plt.ylabel("Normalized RMSE")
plt.title(f"MRPWI Reconstruction Error Comparison\nRe={Re_test}", pad=10)
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
# Plot MRPWI reconstruction errors over interpolated time across stencil sizes for test parameter i.e., Reynolds number

Re_test = 190   # target Reynolds number
t_vec = np.array(sampled_times_test_future, dtype=float)  # ensure numeric time vector
X_true = snapshot_test_future[:, :len(t_vec)]             # true CFD snapshots

fig, ax = plt.subplots(figsize=(8,4))

for Np in [2, 4, 6]:
    # Aligned reconstruction
    X_recon_aligned = recon_results_aligned[Np].real
    abs_error_aligned = np.linalg.norm(X_true - X_recon_aligned, axis=0)
    rel_error_aligned = abs_error_aligned / np.linalg.norm(X_true, axis=0)
    ax.plot(t_vec, rel_error_aligned, lw=2, label=f"MRPWI $N_p$ = {Np}")

    # Unaligned reconstruction
    #X_recon_unaligned = recon_results_unaligned[Np].real
    #abs_error_unaligned = np.linalg.norm(X_true - X_recon_unaligned, axis=0)
    #rel_error_unaligned = abs_error_unaligned / np.linalg.norm(X_true, axis=0)
    #ax.plot(t_vec, rel_error_unaligned, '--', lw=1.5, label=f"Unaligned Np={Np}")

ax.set_xlabel("$t$ (Time in seconds)")
ax.set_ylabel("Relative $L^2$ Error")
ax.set_title(f"MRPWI Reconstruction Error Over Time \n(Re={Re_test})")
ax.grid(True, alpha=0.6)
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Plot flow comparison: true vs interpolated DMD for test parameter i.e., Reynolds number

Re = Re_test
t_start, t_end = 18, 19
granularity = 10              # plot every 'granularity' time steps

Np_vis = 6 # stencil size for visualization

cmap = sns.color_palette("icefire", as_cmap=True)

# Time vector
time_vec = np.array(sampled_times_test_future, dtype=float)
mask = (time_vec >= t_start) & (time_vec <= t_end)
idx = np.where(mask)[0][::granularity]

# Spatial setup
coords = coords_test
num_points = num_points_test
tri = mtri.Triangulation(coords[:, 0], coords[:, 1])

# True snapshots
X_true = snapshot_test_future[:, :len(time_vec)]

# PROM reconstructions

X_recon_aligned = recon_results_aligned[Np_vis]
X_recon_unaligned = recon_results_unaligned[Np_vis]

# Global magnitude limits
mag_true_all = np.sqrt(np.real(X_true[:num_points])**2 + np.real(X_true[num_points:])**2)
mag_aligned_all = np.sqrt(np.real(X_recon_aligned[:num_points])**2 + np.real(X_recon_aligned[num_points:])**2)
vmin_vel = min(mag_true_all.min(), mag_aligned_all.min())
vmax_vel = max(mag_true_all.max(), mag_aligned_all.max())

# Residual symmetric scaling
resid_all = mag_true_all - mag_aligned_all
vmax_resid = np.max(np.abs(resid_all))

# Shared ScalarMappables
sm_shared = ScalarMappable(norm=Normalize(vmin=vmin_vel, vmax=vmax_vel), cmap=cmap)
sm_shared.set_array([])
fixed_ticks_main = np.linspace(vmin_vel, vmax_vel, 10)

sm_resid = ScalarMappable(norm=Normalize(vmin=-vmax_resid, vmax=vmax_resid), cmap=cmap)
sm_resid.set_array([])
fixed_ticks_resid = np.linspace(-vmax_resid, vmax_resid, 10)

# Plotting
n_rows = len(idx)
fig, axes = plt.subplots(n_rows, 3, figsize=(14, 3 * n_rows))
fig.suptitle(f"Flow Comparison — True vs MRPWI Reconstructions\n($Re$ = {Re}, Np={Np_vis})",
             fontsize=18, y=0.94)

if n_rows == 1:
    axes = np.expand_dims(axes, axis=0)

for row, k in enumerate(idx):
    U_true = X_true[:, k]
    U_aligned = X_recon_aligned[:, k]

    ux_true, uy_true = np.real(U_true[:num_points]), np.real(U_true[num_points:])
    ux_aligned, uy_aligned = np.real(U_aligned[:num_points]), np.real(U_aligned[num_points:])

    mag_true = np.sqrt(ux_true**2 + uy_true**2)
    mag_aligned = np.sqrt(ux_aligned**2 + uy_aligned**2)
    residual = mag_true - mag_aligned

    t_val = time_vec[k]

    # True field
    ax_true = axes[row, 0]
    ax_true.tricontourf(tri, mag_true, levels=200, cmap=cmap, vmin=vmin_vel, vmax=vmax_vel)
    ax_true.add_patch(Circle((0.2, 0.2), 0.05, color='black', zorder=10))
    ax_true.set_aspect('equal'); ax_true.set_xticks([]); ax_true.set_yticks([])
    for spine in ax_true.spines.values(): spine.set_visible(False)
    fig.colorbar(sm_shared, ax=ax_true, shrink=0.75, pad=0.02, ticks=fixed_ticks_main)
    ax_true.set_ylabel(f"$t$ = {t_val:.2f} s", fontsize=15, rotation=90, labelpad=25,
                       bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=2))

    # Aligned field
    ax_aligned = axes[row, 1]
    ax_aligned.tricontourf(tri, mag_aligned, levels=200, cmap=cmap, vmin=vmin_vel, vmax=vmax_vel)
    ax_aligned.add_patch(Circle((0.2, 0.2), 0.05, color='black', zorder=10))
    ax_aligned.set_aspect('equal'); ax_aligned.set_xticks([]); ax_aligned.set_yticks([])
    for spine in ax_aligned.spines.values(): spine.set_visible(False)
    fig.colorbar(sm_shared, ax=ax_aligned, shrink=0.75, pad=0.02, ticks=fixed_ticks_main)

    # Residual field
    ax_res = axes[row, 2]
    ax_res.tricontourf(tri, residual, levels=200, cmap=cmap, vmin=-vmax_resid, vmax=vmax_resid)
    ax_res.add_patch(Circle((0.2, 0.2), 0.05, color='black', zorder=10))
    ax_res.set_aspect('equal'); ax_res.set_xticks([]); ax_res.set_yticks([])
    for spine in ax_res.spines.values(): spine.set_visible(False)
    fig.colorbar(sm_resid, ax=ax_res, shrink=0.75, pad=0.02, ticks=fixed_ticks_resid)

# Column titles only once
axes[0, 0].set_title("True Magnitude", fontsize=15, pad=12,
                     bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=2))
axes[0, 1].set_title("Interpolated Reconstruction", fontsize=15, pad=12,
                     bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=2))
axes[0, 2].set_title(r"Residual $(U_{\mathrm{True}} - U_{\mathrm{Reconstruction}})$", fontsize=15, pad=12,
                     bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=2))

plt.tight_layout(rect=[0, 0.03, 1, 0.95], h_pad=1)
plt.show()
